In [5]:
import torchvision.models as models
import torch
import numpy as np
import paddle
from config_cvt.default import get_config
from cvt import generate_model



In [6]:

config=get_config('./config_cvt/config.yaml')
paddle_model = generate_model(config)
paddle_model.eval()

torch_model= models.squeezenet1_1(pretrained=False)
torch_model.load_state_dict= torch.load('CvT-13-224x224-IN-1k.pth',map_location=torch.device('cpu'))


x = np.random.randn(2, 3, 224, 224).astype('float32')
x_paddle = paddle.to_tensor(x)
x_torch = torch.Tensor(x).to('cpu')





merging config from ./config_cvt/config.yaml


In [7]:
out_paddle = paddle_model(x_paddle)
out_paddle = out_paddle.cpu().numpy()
out_paddle

vit:
Tensor(shape=[2, 3, 224, 224], dtype=float32, place=CPUPlace, stop_gradient=True,
       [[[[-0.16885179,  0.40238473, -1.34666944, ...,  0.19293404,
           -0.81369996, -0.82610756],
          [ 0.85447770, -0.30102831,  0.07330187, ..., -0.84217000,
           -0.30881071,  1.59243715],
          [ 0.11884414,  1.70329022,  2.23914337, ..., -0.05424035,
           -1.69417751,  0.54342538],
          ...,
          [-0.24503516,  1.35169673,  0.08933870, ..., -0.29841197,
            1.49583459, -0.06557891],
          [-1.16534579,  0.52506983, -0.70402628, ..., -0.35685250,
           -0.76199239, -0.38153785],
          [ 0.55546528,  2.02862501,  0.11956606, ..., -1.19012558,
           -0.24867114,  0.27797931]],

         [[ 2.00678706, -1.40297151, -0.05817574, ...,  0.93304634,
           -1.82151020, -0.10884593],
          [-0.62578952,  1.57058489,  1.10239959, ...,  0.04242301,
            1.63879812,  0.24371265],
          [ 0.18387657,  0.62929887,  0.59669733

array([[ 0.13462031,  0.21404397,  0.04330602, ...,  0.04111252,
        -0.19077358,  0.77192754],
       [ 0.1531689 ,  0.19592759,  0.11725008, ...,  0.05370414,
        -0.15468869,  0.7874947 ]], dtype=float32)

In [8]:
out_torch = torch_model(x_torch)
out_torch = out_torch.data.cpu().numpy()
out_torch

array([[0.906103  , 0.46645024, 0.10177382, ..., 0.84817123, 0.15589607,
        0.31679502],
       [0.95092785, 0.47769   , 0.1366985 , ..., 0.8356131 , 0.1368592 ,
        0.28241128]], dtype=float32)